In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read data 
#sample = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv")
#train = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train = pd.read_csv("/kaggle/input/detect-sleep-states/detect-sleep-states/train-series_v1/train_events.csv")
#train_series = pd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet",chunk_size=1000000)
#test_series = pd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet")
# train_series = parquet_by_chunks_to_df("/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet")
# test_series = pd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet")

In [ ]:
train

In [ ]:
serie_id = 'fe90110788d2'
train_series  = {}
train_series[serie_id] = pd.read_csv("/kaggle/input/detect-sleep-states/detect-sleep-states/train-series_v1/"+serie_id+".csv")



In [ ]:
new_column_len = len(train_series[serie_id])

In [ ]:
encode_list = {
    "unknown":0,
    "awake":1,
    "onset":2,
    "sleep":3,
    "wakeup":4
}

def encode_events(df,encode_list):
    df_copy = df.copy()
    df_copy["event"] = df_copy["event"].map(encode_list)    
    return df_copy

def decode_events(df,encode_list):
    df_copy = df.copy()
    decoded_list = {v: k for k, v in encode_list.items()}
    df_copy["event"] = df_copy["event"].map(decoded_list)    
    return df_copy
    

In [ ]:
try:
    del df_test
except:
    pass
df_test = train[train["series_id"]==serie_id]
df_test

In [ ]:
encoded_df = encode_events(df_test,encode_list)



In [ ]:
decoded_df = decode_events(encoded_df,encode_list)
decoded_df

In [ ]:
# 
# every step is recorded within 5s 
#time_window
record_interval = 5 #[s]
key = 12
# for key in keys:
time_window = key*60*60/record_interval 

time_slices = [(time_window*(i/4),time_window*(4-i/4)) for i in range(1,4)]

valid_range = int(time_window/4)

In [ ]:
for idx, *row in encoded_df.itertuples():
    event = row[2]
    step = row[3]
    timestamp = row[4]
    
    
    try: 
        print(type(event))
    except:
        pass

In [ ]:






def step_segmentation_list(df,valid_range,encoded_list, list_len):
    """
    df = dataframe with noted nights 
    valid_range = valid range of steps to be taken into account when the previous/next step is NaN (not noted
    segmentation_values - every event has different segmentation value 
    
    returns list of segmentation values to be added to dataframe
    )"""
    step_seg_list = np.zeros(shape =(list_len,),dtype= int)
    last_step =  -1
    last_timestamp_na = False
    last_event = -1
    first_iter_completed= False
    # e.g. to access the `exchange` values as in the OP
    for idx, *row in df.itertuples():
        event = row[2]
        try:
            step = int(row[3])
        except:
            step = row[3]
        timestamp = row[4]
        try: 
            timestamp_na = np.isnan(timestamp)
        except:
            timestamp_na = False
        # check if value is accepted 
        if event != encoded_list["wakeup"] and event != encoded_list["onset"]:
            raise ValueError(f"last_event on step {last_step} is not wakeup or onset. Value is {last_event}")

        # check if this is first iteration
        if first_iter_completed == False:
            if timestamp_na:
                last_timestamp_na = True
            else:
                step_seg_list[step] = event
            last_step = step
            last_event = event 
            first_iter_completed = True
            continue
        
        # if range should not be monitored, just continue            
        if timestamp_na and last_timestamp_na :
            last_timestamp_na = True
            
        # if current timestamp shouldn't be monitored, update valid_range on the left side (from last_step (last_step not included))        
        elif timestamp_na and not last_timestamp_na:
            min_range= last_step+1
            max_range=last_step+1+valid_range
            
            if last_event == encoded_list["wakeup"]:
                step_seg_list[min_range:max_range] = encoded_list["awake"]
            elif last_event == encoded_list["onset"]:
                step_seg_list[min_range:max_range] = encoded_list["sleep"] 
            
            last_timestamp_na = True

        # if last timestamp shouldn't be monitored, update valid_range on the right side (up to current_step(current_step included))        
        elif not timestamp_na and last_timestamp_na: 
            min_range= step-valid_range
            max_range=step
            print(min_range,max_range)
            
            if event == encoded_list["wakeup"]:
                step_seg_list[min_range:max_range] = encoded_list["sleep"]
            elif event == encoded_list["onset"]:
                step_seg_list[min_range:max_range] = encoded_list["awake"]
            
            step_seg_list[step] = event
            last_timestamp_na = False
        
        # both timestamps valid - update the range from last_step(last_step not included) to current_step (included)
        else:
            min_range= last_step+1
            max_range=step
            print(min_range,max_range)
            
            if event == encoded_list["wakeup"]:
                step_seg_list[min_range:max_range] = encoded_list["sleep"]
            elif event == encoded_list["onset"]:
                step_seg_list[min_range:max_range] = encoded_list["awake"]
            
            step_seg_list[step] = event
            last_timestamp_na = False
        
        #update last values to next iteratation
        last_step = step
        last_event = event 
      
    return step_seg_list
        
            
        

In [ ]:
step_seg_list = step_segmentation_list(encoded_df,valid_range,encode_list,new_column_len )

In [ ]:
step_seg_list

In [ ]:
train_concat = train_series[serie_id]
train_concat["event"] = step_seg_list
train_concat

In [ ]:
train_concat
concat_series = train_concat[train_concat['event'] != 0]


In [ ]:
try:
    del train_concat
except:
    pass
train_concat = pd.merge(left = train_series[serie_id], right = df_test,how="outer")
# remove data without timestamps 
train_concat = train_concat[train_concat["timestamp"].notna()]
train_concat




In [ ]:
def create_slices(concat_series, time_window):
    slices = []
    #concat_series = concat_series[concat_series["timestamp"].notna()]
    concat_series = concat_series[concat_series['event'] != 0]
    for i in range(int(len(concat_series)/time_window)):
        slice = concat_series[(concat_series['step'] < time_window+time_window*i) & (concat_series['step']  >= time_window*i) ]    
        slices.append(slice)
    return slices

slices = create_slices(train_concat,time_window)

In [ ]:
slices[1]

In [ ]:
train_concat

In [ ]:
# import os
# os.chdir(r'kaggle/working')'
filename = 'concat_'+serie_id + '.csv'
train_concat.to_csv(filename)

from IPython.display import FileLink
FileLink(filename)


In [ ]:
df_test = train[train["series_id"]==serie_id]
df_test

In [ ]:
train_series[serie_id]

In [ ]:
train[train["series_id"]=="038441c925bb"]

In [ ]:
print(train.info())
print(train_series.info())
print(test_series.info())

In [ ]:
series = train["series_id"].unique()
series

In [ ]:
# remove data without timestamps 
trainNotNan = train["timestamp"].notna()
trainNotNan = train[trainNotNan]
# check typical series
trainNotNan[trainNotNan["series_id"].str.contains("5ffd5e1e81ac")]


In [ ]:
train_series.info

In [ ]:
train_series[train_series["series_id"].str.contains("038441c925bb").str.contains["2018-04-14"]]

In [ ]:
train_series[train_series["series_id"].str.contains("2018-04-14")

In [ ]:
train_series[train_series.where("series_id" == "038441c925bb") ]

In [ ]:
train_series.where("series_id" == "038441c925bb") 